**==================TASK 1 : TF-IDF-KNN====================**

In [ ]:
! pip install cltk -q -U
! pip install demoji

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 845.0/845.0 kB 12.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 802.5/802.5 kB 34.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.5/50.5 kB 3.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.3/184.3 kB 11.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.7/193.7 kB 12.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 240.9/240.9 kB 16.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.9/42.9 kB 3.1 MB/s eta 0:00:00


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
import io
import re
import demoji
from sklearn.neighbors import KNeighborsClassifier
from sklearn import metrics
from sklearn.metrics import confusion_matrix,classification_report
import pandas as pd
import numpy as np

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving hindi_test.csv to hindi_test.csv
Saving hindi_train_val.csv to hindi_train_val.csv
Saving stopwords.txt to stopwords.txt


In [ ]:
data_csv = 'hindi_train_val.csv'
stop_csv = 'stopwords.txt'
stop_words = 'stopwords'
df = pd.read_csv(io.BytesIO(uploaded[data_csv]))
df_test = pd.read_csv(io.BytesIO(uploaded['hindi_test.csv']))
stop_words = pd.read_csv(io.BytesIO(uploaded[stop_csv]))
stop_words.columns = ['stopwords']
stop_words = stop_words['stopwords'].tolist()
print(df)
print(stop_words)

       label                                               text
0          0                      भीड़ में  बहुत  हब्सी मिलेंगे
1          0          साले बेवकूफ अपनी मां मक्खियां  तो हटा दें
2          0                   बुर देदो तो मुह में लंड ले लो तो
3          0               कुत्ता वहा है चिल्ला तू क्यों रहा है
4          1          चाय नहीं पीता हूं मैं इसी को छोड़ दिया ok
...      ...                                                ...
20178      0             और छोटे भी क्यु पहनते हो, कहा जरूरत है
20179      1             कैसी हो सोना क्या आप हमसे दोस्ती करोगी
20180      1          इस से ये पता चलता है अब चुनाव आ रहा है🙄🙄🙄
20181      1                    खेत जोते हुए किसान भैंस के साथो
20182      0  काश आगर यह इतना दिमाग पढाई में लगायें तो शायद ...

[20183 rows x 2 columns]
['मुझको', 'मेरा', 'अपने आप को', 'हमने', 'हमारा', 'अपना', 'हम', 'आप', 'आपका', 'तुम्हारा', 'अपने आप', 'स्वयं', 'वह', 'इसे', 'उसके', 'खुद को', 'कि वह', 'उसकी', 'उसका', 'खुद ही', 'यह', 'इसके', 'उन्होने', 'अपने'

In [ ]:
def drop_stopwords(sent):
    #print(sent)
    ret = ""
    words=[]
    start_index = 0
    for i in range(len(sent)):
      if sent[i] == ' ':
        words.append(sent[start_index:i])
        start_index = i + 1
    words.append(sent[start_index:])
    space=' '
    for word in words:
        if word not in stop_words:
            ret = ret + space + word
    return ret

In [ ]:
X = df.drop(['label'], axis = 1)
X = X.to_numpy()
y = df['label']
y = y.to_numpy()
X_temp = []

X_test = df_test.drop(['label'], axis = 1)
y_test = df_test['label']

X_temp_test = []

X_test = X_test.to_numpy()
y_test = y_test.to_numpy()

set_size=1
punctuations = ['!', '(', ')', '-', '[', ']', '{', '}', ';', ':', '\'', '\"', '\\', ',', '<', '>', '.', '/', '?', '@', '#', '$', '%', '^', '&', '*', '_', '~']
data_len = len(X)

for i in range(set_size-1, data_len):
    temp = np.array_str(X[i])
    curr_sent = ""
    for char in temp:
        if char not in punctuations:
            curr_sent +=char
    curr_sent = re.sub(r'\b\d+\b', ' ', curr_sent)
    curr_sent = re.sub(r'[^\u0900-\u097F|\d+|\w+]', ' ', curr_sent)
    curr_sent = drop_stopwords(curr_sent)
    X[i] = curr_sent

for i in range(0, len(X_test)):
    curr_sent = ""
    temp = np.array_str(X_test[i])
    for char in temp:
        if char not in punctuations:
            curr_sent = curr_sent + char
    #curr_sent = re.sub(r"[a-zA-Z0-9]", "", curr_sent)
    curr_sent = re.sub(r'[^\u0900-\u097F|\d+|\w+]', ' ', curr_sent)
    curr_sent = re.sub(r'\b\d+\b', ' ', curr_sent)
    curr_sent = drop_stopwords(curr_sent)
    X_test[i] = curr_sent
print(X)

[[' भीड़   हब्सी मिलेंगे']
 [' साले बेवकूफ मां मक्खियां  हटा दें']
 [' बुर देदो मुह लंड ले लो']
 ...
 [' पता चलता चुनाव आ   ']
 [' खेत जोते किसान भैंस साथो']
 [' काश आगर इतना दिमाग पढाई लगायें शायद जाकर डॉक्टर इंजीनियर बन सक्ति है।']]


In [ ]:
temp = []
set_size=1
data_len = len(X)
Range = range(0, data_len)
for i in Range:
    temp.append(X[i][set_size-1])
X = temp
print(X)

temp_test = []
for i in range(0, len(X_test)):
    temp_test.append(X_test[i][0])
X_test = temp_test
print(X_test)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [ ]:
vectorizer = TfidfVectorizer(lowercase=False)
X_vector = vectorizer.fit_transform(X)
testing_size_data = 0.2
X_train, X_temp, y_train, y_temp = train_test_split(X_vector, y, test_size=testing_size_data, random_state=42)
X_test = X_temp
y_test = y_temp

In [ ]:

from sklearn.metrics import accuracy_score, f1_score
from sklearn.metrics import accuracy_score, f1_score

classifier = KNeighborsClassifier(n_neighbors=43)
classifier.fit(X_train, y_train)
y_pred = classifier.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f'Train Accurcy: {accuracy:.4f}')
f1 = f1_score(y_test, y_pred)
print(f'F1 Score: {f1:.4f}')
print(classification_report(y_test,y_pred))
Range_k = range(50,51)
scores = {}
scores_list = []

for k in Range_k:
   classifier = KNeighborsClassifier(n_neighbors=k)
   classifier.fit(X_train, y_train)
   y_pred = classifier.predict(X_test)
   scores[k] = metrics.accuracy_score(y_test,y_pred)
   scores_list.append(metrics.accuracy_score(y_test,y_pred))

print("Test accuracy : ")
print(scores_list[0])

Train Accurcy: 0.6349
F1 Score: 0.5312
              precision    recall  f1-score   support

           0       0.62      0.81      0.70      2140
           1       0.67      0.44      0.53      1897

    accuracy                           0.63      4037
   macro avg       0.64      0.62      0.62      4037
weighted avg       0.64      0.63      0.62      4037

Test accuracy : 
0.6222442407728511


===============INTERSECTION FUNCTION======================

In [ ]:
def intersection_fun(X_1, X_2):
    common = 0
    Range_K = range(0, len(X_1))
    set_size = 0
    for i in Range_K:
        for j in range(set_size, len(X_2)):
            if(X_1[i] == X_2[j]):
                common =common + 1

    return common

In [ ]:
df = pd.read_csv(io.BytesIO(uploaded[data_csv]))
df_test = pd.read_csv(io.BytesIO(uploaded['hindi_test.csv']))

X = df.drop(['label'], axis = 1)
y = df['label']
X = X.to_numpy()
X_temp = []
y = y.to_numpy()

set_size=1
X_test = df_test.drop(['label'], axis = set_size)
X_temp_test = []
y_test = df_test['label']


X_test = X_test.to_numpy()
y_test = y_test.to_numpy()
###Calling Intersection Function
intersection_fun(X_test, X)

0